In [1]:
import pandas as pd
import json
import sklearn.svm as svm
import numpy as np

In [54]:
## Change the name to get your path
user = 'Gus'
file1 = 'AG01-01.csv'
file2 = 'GTH01-01.csv'

In [55]:
def load_Data(user, filename):
   data = json.load(open('dir.json'))
   dir = data[user] + filename
#    data = pd.read_csv(dir, header=2)
   return dir

In [148]:
data = pd.read_csv(load_Data(user, file1), header=0)
data['start.time'] = pd.to_datetime(data['start.time'])
data['Class'] = np.random.randint(2, size=len(data))
## selecting the times when the ground truth was observed
observedData = data[(data['start.time'] >= '2017-10-03 18:45:00') & (data['start.time'] <= '2017-10-03 20:47:00')]
observedData['index'] = range(len(observedData))

/Users/gusmoir/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [149]:
gt = pd.read_csv(load_Data(user, file2), header=0)
gt['index'] = gt['time']
# gt.head()

In [150]:
observedData = pd.merge(observedData,gt[['index','coding']])

In [157]:
test = observedData[6000:]
train = observedData[:6000]
## this might be an issue since there is ordinality and that makes things weird.  The later parts of this trial
## were more likely to be sedentary.
X = train[['mean.vm','sd.vm','mean.ang','sd.ang','p625','dfreq','ratio.df']]
y = train['coding']

In [158]:
clf = svm.SVC(cache_size=7000)
clf.fit(X, y)

SVC(C=1.0, cache_size=7000, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [159]:
clf.predict([[1,.0004,1,1,1,9,1],[.5,.002,.1,3,1,.5,1]])

array(['non-sedentary', 'non-sedentary'], dtype=object)

In [160]:
X = test[['mean.vm','sd.vm','mean.ang','sd.ang','p625','dfreq','ratio.df']]
y = test['coding']

In [161]:
## get accuracy of the svm
clf.score(X,y)

0.31225296442687744

In [146]:
loc = 3
a = [test.iloc[loc]["mean.vm"],test.iloc[loc]["sd.vm"],test.iloc[loc]["mean.ang"],test.iloc[loc]["sd.ang"],test.iloc[loc]["p625"],test.iloc[loc]["dfreq"],test.iloc[loc]["ratio.df"]]
print(" Predicted ", clf.predict([a])[0],'\n',"Actual ",test.iloc[loc]["coding"])

 Predicted  sedentary 
 Actual  non-sedentary


In [12]:
observedData.head(10)


,index,mean.vm,sd.vm,mean.ang,sd.ang,p625,dfreq,ratio.df,start.time,Class,coding
0,0,0.987755,0.005804,-23.779800,0.654577,0.165556,2.051282,0.141226,2017-10-03 18:45:00,0,non-sedentary
1,1,0.985573,0.004706,-23.380848,0.455990,0.122440,1.025641,0.106751,2017-10-03 18:45:00,1,non-sedentary
2,2,1.011537,0.111933,-23.051432,8.022191,0.110025,4.102564,0.081356,2017-10-03 18:45:00,1,non-sedentary
3,3,0.992961,0.141553,-24.960066,7.597464,0.137500,2.051282,0.095910,2017-10-03 18:45:00,0,non-sedentary
4,4,0.995071,0.035139,-23.534394,2.397515,0.281809,1.025641,0.178779,2017-10-03 18:45:00,1,non-sedentary
5,5,0.995704,0.019732,-24.957561,1.152612,0.174133,2.051282,0.106816,2017-10-03 18:45:00,1,non-sedentary
6,6,1.092518,0.515226,-22.976681,11.143662,0.126769,3.076923,0.081500,2017-10-03 18:45:00,0,non-sedentary
7,7,1.001879,0.124899,-45.539832,19.231341,0.359780,2.051282,0.234417,2017-10-03 18:45:00,1,non-sedentary
8,8,1.233121,0.513984,-41.604872,21.224018,0.315810,2.051282,0.177017,2017-10-03 18:45:00,0,non-sedentary
9,9,0.982744,0.113845,-25.492668,11.009674,0.120192,2.051282,0.111920,2017-10-03 18:45:00,0,non-sedentary


In [92]:
## what if we tried a neural net for fun?
from sklearn.neural_network import MLPClassifier
n_net = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(2000, 1000), random_state=1)
n_net.fit(X,y)

In [93]:
n_net.fit(X,y)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(2000, 1000), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [94]:
## test accuracy of neural net
np.sum(n_net.predict(X) == y) / len(X)

0.6799338934031125